In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.tokens import Doc, Span

import numpy as np
import os

# Monta Google Drive (solo per Colab)
from google.colab import drive
drive.mount('/content/drive')

# Definisci la directory dove sono salvati i file
drive_path = '/content/drive/MyDrive/Colab Notebooks/Bert Trial 2'
file_path = os.path.join(drive_path, 'file1.ann')

# Funzione per creare il DataFrame
def create_dataframe(content):
    with open(content, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    data = [line.split() for line in lines]
    df = pd.DataFrame(data, columns=['token', 'tag'])
    df = df.drop(index=df.index[0])
    return df

# Carica il file con il percorso specifico
df_file1 = create_dataframe(file_path)

# Funzione per pulire il DataFrame
def pulisci_df(df):
    def is_arabic(word):
        if word is None or pd.isnull(word):  # Controlla per None o NaN
            return True
        word = str(word)
        return any('\u0600' <= char <= '\u06FF' for char in word)

    mask = df['token'].apply(is_arabic)
    return df[~mask]  # Tieni solo le righe dove la condizione è False

# Pulisci e reimposta l'indice del DataFrame
df_file1 = pulisci_df(df_file1)
df_file1 = df_file1.reset_index(drop=True)

# Stampa il DataFrame risultante
print(df_file1.head())



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       token tag
0  Operation   O
1      Steel   O
2    Curtain   O
3          (   O
4     Arabic   O


In [ ]:
## ------------------------------------------------------------------------------
#CREATE A SPACY DF

def pulisci_file(df, column_name):
    result = df[column_name].str.cat(sep=' ')
    return result

file1_text = pulisci_file(df_file1, 'token')

doc1 = nlp(file1_text)

##
row_to_insert1 = [".", "O"]
row_to_insert2 = [".", "I-Location"]
row_to_insert3 = [".", "I-Quantity"]
row_to_insert4 = [".", "I-Temporal"]
row_to_insert5 = [".", "I-Person"]

# The indexes where rows need to be inserted
indexes_to_insert = {239, 946, 981, 2165, 2298 #I-Location
                     , 3108, #I-Location,
                     3487, 3674, #I_quantity,
                     3929, 4207, 7248,
                     7988, 8201, #I_quantity,
                     11567, 11741, 13590, #I_quantity
                     13593, #I_quantity
                     13943,#O
                     14364, #I_quantity
                     14830,#I_Location
                     16391, #I_Temporal
                     16592, #I_quantity
                     17136, 18007, 19293}

# Insert the rows and reset index
for index in sorted(indexes_to_insert):
    if index == 13943:
        new_row = pd.DataFrame([row_to_insert1], columns=df_file1.columns)
    elif index in {2298, 3108, 14830}:
        new_row = pd.DataFrame([row_to_insert2], columns=df_file1.columns)
    elif index in {3674, 13590, 13593, 14364, 16592}:
        new_row = pd.DataFrame([row_to_insert3], columns=df_file1.columns)
    elif index == 16391:
        new_row = pd.DataFrame([row_to_insert4], columns=df_file1.columns)
    else:
        new_row = pd.DataFrame([row_to_insert5], columns=df_file1.columns)

    df_file1 = pd.concat([df_file1.iloc[:index], new_row, df_file1.iloc[index:]], ignore_index=True)




In [ ]:
# Funzione per calcolare la frequenza delle classi e sostituire quelle con frequenza minore di 100 con 'O'
def replace_low_frequency_tags(df, label_column='tag', threshold=100, replacement_tag='O'):
    # Calcola la frequenza delle etichette
    frequencies = df[label_column].value_counts()

    # Stampa le frequenze
    print("Frequenze delle etichette originali:")
    print(frequencies)

    # Trova le etichette con frequenza inferiore al threshold
    low_freq_tags = frequencies[frequencies < threshold].index.tolist()

    # Sostituisci le etichette con frequenza inferiore a threshold con 'O'
    df[label_column] = df[label_column].apply(lambda x: replacement_tag if x in low_freq_tags else x)

    # Stampa le nuove frequenze dopo la sostituzione
    new_frequencies = df[label_column].value_counts()
    print("\nFrequenze delle etichette dopo la sostituzione con 'O':")
    print(new_frequencies)

    return df, new_frequencies

# Usa la funzione per modificare df_file1
df_file1_modified, new_label_frequencies = replace_low_frequency_tags(df_file1, label_column='tag')

# Stampa il DataFrame modificato
print("\nDataFrame modificato (prime righe):")
print(df_file1_modified.head())


Frequenze delle etichette originali:
tag
O                      13635
I-Organisation          1528
B-Organisation          1103
I-Location               787
B-Location               644
I-Person                 620
B-Person                 411
I-Temporal               226
B-Temporal               177
B-Quantity               150
I-DocumentReference      146
I-Quantity                99
B-Weapon                  81
I-Weapon                  70
I-MilitaryPlatform        64
B-DocumentReference       50
B-MilitaryPlatform        47
I-Money                   36
B-Nationality             34
B-Money                   20
I-Nationality              2
Name: count, dtype: int64

Frequenze delle etichette dopo la sostituzione con 'O':
tag
O                      14138
I-Organisation          1528
B-Organisation          1103
I-Location               787
B-Location               644
I-Person                 620
B-Person                 411
I-Temporal               226
B-Temporal               177
B-

In [ ]:
def extract_entities(df, tag_column="tag"):
    entities = []
    current_entity = None

    for index, row in df.iterrows():
        tag = row[tag_column]

        if tag.startswith("B-"):  # Begin a new entity
            if current_entity:  # Save the previous entity
                entities.append(current_entity)
            entity_type = tag[2:]  # Extract the entity type
            current_entity = {"type": entity_type, "start": index, "end": index + 1}

        elif tag.startswith("I-") and current_entity:  # Inside an entity
            entity_type = tag[2:]
            if current_entity["type"] == entity_type:  # Ensure it's the same entity type
                current_entity["end"] = index + 1  # Extend the current entity

        else:  # Outside any entity
            if current_entity:  # Save the current entity
                entities.append(current_entity)
                current_entity = None

    # Add the last entity if it exists
    if current_entity:
        entities.append(current_entity)

    return entities

# Extract entities
custom_entities = extract_entities(df_file1_modified)

doc1.ents = [Span(doc1, entity["start"], entity["end"], label=entity["type"]) for entity in custom_entities]
doc1.set_ents(doc1.ents, default="outside")

##
data = []
for token in doc1:
    data.append([
        getattr(token, "text"),      # Token text
        getattr(token, "lemma_", None),   # Lemma
        getattr(token, "pos_", None),     # Part-of-speech tag
        getattr(token, "dep_", None),     # Dependency label
        getattr(token.head, "text", None),# Head text
        f"{token.ent_iob_}-{token.ent_type_}" if token.ent_type_ else "O"
    ])


def join_and_clear(df, col_name, row1, row2):
    df.at[row1, col_name] = f"{df.at[row1, col_name]} {df.at[row2, col_name]}"
    df.loc[row2] = [None] * len(df.columns)

df = pd.DataFrame(data, columns=["token", "Lemma", "POS", "DEP", "Head", "tag"])

#CREATE A DF WITH THE SENTENCES AND MATCHES

data = []

for sent in doc1.sents:
    sentence_text = sent.text
    iob_tags = []
    for token in sent:
        if token.ent_iob_ == "O":
            iob_tags.append(token.ent_iob_)
        else:
            iob_tags.append(token.ent_iob_ + "-" + token.ent_type_)
    iob_tags_str = ' '.join(iob_tags)
    data.append({"sentence": sentence_text, "IOB_tag": iob_tags_str})

df1 = pd.DataFrame(data)

# 1.1) TRAINING AND VALIDATION DIVISION

In [ ]:
from sklearn.model_selection import train_test_split

# Supponiamo che sentences e tags siano le tue liste
train_sentences, val_sentences, train_tags, val_tags = train_test_split(
    df1["sentence"], df1["IOB_tag"], test_size=0.25, random_state=42
)

# 1.1) TOKENIZATION AND TAG ADJUSTMENT
from transformers import AutoTokenizer

# Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

from transformers import AutoTokenizer

def tokenize_and_align_labels(sentences, tags, tokenizer):
    tokenized_inputs = []
    aligned_labels = []

    for sentence, tag_seq in zip(sentences, tags):
        # Split sentence into words
        sentence_tokens = sentence.split()
        tokenized_sentence = tokenizer(
            sentence_tokens,
            is_split_into_words=True,
            truncation=True,
            padding="max_length"  # Optional: Add padding
        )

        word_ids = tokenized_sentence.word_ids()  # Map sub-tokens to words
        aligned_tags = []
        current_word_idx = None

        # Align tags to tokens
        for word_idx in word_ids:
            if word_idx is None:
                aligned_tags.append("O")  # Padding, CLS, or SEP tokens
            else:
                if word_idx != current_word_idx:
                    aligned_tags.append(tag_seq.split()[word_idx])
                else:
                    aligned_tags.append(tag_seq.split()[word_idx])
                current_word_idx = word_idx

        tokenized_inputs.append({
            "input_ids": tokenized_sentence["input_ids"],
            "attention_mask": tokenized_sentence["attention_mask"]
        })
        aligned_labels.append(aligned_tags)

    return tokenized_inputs, aligned_labels


# Tokenizza e riallinea per train e validation
train_tokenized, train_labels = tokenize_and_align_labels(train_sentences, train_tags, tokenizer)
val_tokenized, val_labels = tokenize_and_align_labels(val_sentences, val_tags, tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# 1.2) PROPER DATASET FOR TRAINING

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

from datasets import Dataset
import numpy as np

# Creazione del dataset di addestramento (train)
train_dataset = Dataset.from_dict({
    'text': train_sentences,  # Frasi di addestramento
    'labels': train_labels  # Etichette di addestramento riallineate
})

# Creazione del dataset di validazione (validation)
val_dataset = Dataset.from_dict({
    'text': val_sentences,  # Frasi di validazione
    'labels': val_labels  # Etichette di validazione riallineate
})


#1.3) CONVERTING LABELS TO NUMERIC IN DATASETS

In [ ]:
def build_tag2unique(df, tag_column="IOB_tag"):
    """
    Costruisce un dizionario che mappa ciascuna IOB tag a un valore numerico univoco.

    Parametri:
        df (pd.DataFrame): DataFrame contenente le colonne con le IOB tags.
        tag_column (str): Nome della colonna contenente le IOB tags.

    Ritorna:
        dict: Dizionario con tags come chiavi e valori numerici univoci.
    """
    # Estrai tutte le tags in una lista unica
    all_tags = df[tag_column].str.split().sum()  # Dividi ogni stringa di tag in parole
    unique_tags = set(all_tags)  # Mantieni solo le tag uniche

    # Crea il dizionario tag2unique mappando ogni tag a un valore numerico
    tag2unique = {tag: idx for idx, tag in enumerate(sorted(unique_tags))}

    return tag2unique


# Esempio di utilizzo
tag2unique = build_tag2unique(df1, tag_column="IOB_tag")
print("Dizionario tag2unique:", tag2unique)

def map_labels_to_numeric(example, tag2unique):
    # Mappa ogni etichetta a un numero usando il dizionario tag2unique
    example['labels'] = [tag2unique[tag] for tag in example['labels']]
    return example

# Applicare la mappatura a tutte le etichette nel dataset usando il metodo `.map()`
train_dataset = train_dataset.map(lambda x: map_labels_to_numeric(x, tag2unique))
val_dataset = val_dataset.map(lambda x: map_labels_to_numeric(x, tag2unique))


Dizionario tag2unique: {'B-Location': 0, 'B-Organisation': 1, 'B-Person': 2, 'B-Quantity': 3, 'B-Temporal': 4, 'I-Location': 5, 'I-Organisation': 6, 'I-Person': 7, 'I-Temporal': 8, 'O': 9}


Map:   0%|          | 0/595 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

# 1.4) TENSORS FOR INPUTS

In [ ]:
train_input_ids = [item["input_ids"] for item in train_tokenized]
train_attention_mask = [item["attention_mask"] for item in train_tokenized]

val_input_ids = [item["input_ids"] for item in val_tokenized]
val_attention_mask = [item["attention_mask"] for item in val_tokenized]

#1.5) PROPER DATASET (ATTENTION MASKS AND INPUT IDS)

In [ ]:

def add_tokenized_columns(dataset, tokenized_sentences):
    # Aggiungi input_ids e attention_mask a ciascun esempio del dataset
    input_ids = [item['input_ids'] for item in tokenized_sentences]
    attention_mask = [item['attention_mask'] for item in tokenized_sentences]

    dataset = dataset.add_column('input_ids', input_ids)
    dataset = dataset.add_column('attention_mask', attention_mask)
    return dataset


# Aggiungi le colonne a train_dataset
train_dataset = add_tokenized_columns(train_dataset, train_tokenized)
val_dataset = add_tokenized_columns(val_dataset, val_tokenized)


#2) TRAINING

In [ ]:
pip install datasets

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer
import torch
from datasets import Dataset
import csv
import os
import numpy as np
from tqdm.auto import tqdm  # Barra di progresso
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

drive_path = '/content/drive/My Drive/Colab Notebooks/Bert Trial 2'
train_metrics_file = os.path.join(drive_path, 'train_metrics.csv')
val_metrics_file = os.path.join(drive_path, 'val_metrics.csv')
model_save_path = os.path.join(drive_path, 'final_model')

# Carica tokenizer e modello
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForTokenClassification.from_pretrained(
    model_ckpt,
    num_labels=10,

).to(device)

# Crea data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Funzione per metriche (Accuracy e F1 macro)
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1).flatten()
    labels = labels.flatten()

    # Filtra valori -100
    mask = (labels != -100) & (labels != 9)
    filtered_labels = labels[mask]
    filtered_predictions = predictions[mask]

    accuracy = accuracy_score(filtered_labels, filtered_predictions)
    f1_macro = f1_score(filtered_labels, filtered_predictions, average='macro')
    f1_weighted = f1_score(filtered_labels, filtered_predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }

# Funzione per salvare metriche in file CSV
def save_metrics_to_csv(metrics_dict, file_path):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=metrics_dict.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(metrics_dict)

# Impostazioni di training
training_args = TrainingArguments(
    output_dir=os.path.join(drive_path, 'results'),
    num_train_epochs=4,  # Ridotto da 5 a 3 per limitare il rischio di overfitting
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,  # Ridotto rispetto a 0.1
    logging_dir=os.path.join(drive_path, 'logs'),
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    report_to=None,  # Disabilita log per strumenti esterni
    load_best_model_at_end=True,  # Carica il miglior modello al termine del training
    max_grad_norm=1.0,  # Limita la norma del gradiente
    learning_rate=3e-5  # Learning rate ridotto
)

# Creazione Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.shuffle(seed=42),  # Shuffle dei dati
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Addestra il modello
print("Inizio training...")
for epoch in range(training_args.num_train_epochs):
    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs}")

    # Addestramento del modello
    trainer.train()

    # Log delle metriche durante l'addestramento
    train_results = trainer.evaluate(train_dataset)
    train_metrics = {
        "epoch": epoch + 1,
        "train_loss": train_results['eval_loss'],
        "train_accuracy": train_results.get('eval_accuracy', 'N/A'),
        "train_f1_macro": train_results.get('eval_f1_macro', 'N/A'),
        "train_f1_weighted": train_results.get('eval_f1_weighted', 'N/A')
    }
    save_metrics_to_csv(train_metrics, train_metrics_file)

    # Valutazione su validation set
    val_results = trainer.evaluate(val_dataset)
    val_metrics = {
        "epoch": epoch + 1,
        "eval_loss": val_results['eval_loss'],
        "eval_accuracy": val_results.get('eval_accuracy', 'N/A'),
        "eval_f1_macro": val_results.get('eval_f1_macro', 'N/A'),
        "eval_f1_weighted": val_results.get('eval_f1_weighted', 'N/A')
    }
    save_metrics_to_csv(val_metrics, val_metrics_file)

# Salva il modello finale
print("Salvataggio del modello finale...")
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Modello salvato in: {model_save_path}")

# Report finale
print("Training completato. Metriche finali salvate.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Inizio training...
Epoch 1/4


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,2.095100,1.685977,0.000000,0.000000,0.000000
2,0.208400,0.105394,0.000000,0.000000,0.000000
3,0.078600,0.063884,0.000000,0.000000,0.000000
4,0.057800,0.048395,0.122175,0.045160,0.112741


Epoch 2/4


...

Epoch 3/4


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.023700,0.025590,0.597434,0.434030,0.639920
2,0.021900,0.024200,0.671350,0.527032,0.702989
3,0.020400,0.022417,0.667074,0.586736,0.726737
4,0.018300,0.022123,0.668907,0.635366,0.750168


Epoch 4/4


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.013200,0.020037,0.737324,0.670743,0.792275
2,0.012300,0.019909,0.764814,0.698139,0.812765
3,0.012000,0.020271,0.743433,0.707766,0.806221
4,0.012200,0.021556,0.716555,0.700280,0.796015


Salvataggio del modello finale...
Modello salvato in: /content/drive/My Drive/Colab Notebooks/Bert Trial 2/final_model
Training completato. Metriche finali salvate.
